In [1]:
%pylab inline
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import os
import sys

Populating the interactive namespace from numpy and matplotlib


In [2]:
src_path = os.getcwd().split('/')
src_path = '/'.join(src_path[:src_path.index('src')+1])
sys.path.append(src_path)
from data.dataset import Dataset

In [3]:
ds = Dataset('SUSY_100k.csv')

In [4]:
X = ds.get_x()[0:100]
y = ds.get_y()[0:100]

In [5]:
m = SVC()
str(type(m))

"<class 'sklearn.svm.classes.SVC'>"

In [12]:
class Active_Learner:
    def __init__(self, model, start_size=0.1,end_size=0.25,step_size=0.01):
        if type(model) == type(SVC()):
            self.m = SVC(probability=True)
        else:
            self.m = model
        self.start_size = start_size
        self.end_size = end_size
        self.step_size = step_size
        
    def fit(self, X, y):
        X_train, X_unlabeled, y_train, y_unlabeled = train_test_split(X, y, train_size=0.10, random_state=42)
        
        while(len(y_train) <= len(y)*self.end_size):
            self.m.fit(X_train,y_train)
            y_unlabeled_hat = self.m.predict_proba(X_unlabeled)
            
            #select points with low confidence (class probabilty close to 0.5)
            lowest_conf_idx = np.flip(np.argsort(abs(0.5-y_unlabeled_hat[:,0])),axis=0)

            #add points of least confidence to training set
            X_train = np.concatenate((X_train,X_unlabeled[:int(len(y)*self.step_size)]),axis=0)
            y_train = np.concatenate((y_train,y_unlabeled[:int(len(y)*self.step_size)]),axis=0)
            
            #remove these points from "unlabeled" set
            mask = np.ones(len(y_unlabeled), dtype=bool)
            mask[lowest_conf_idx[0:int(len(y)*self.step_size)]] = False
            y_unlabeled = y_unlabeled[mask]
            X_unlabeled = X_unlabeled[mask]
            
    def predict(self,X):
        return self.m.predict(X)

In [13]:
sv = SVC()
al = Active_Learner(sv)
al.fit(X,y)

In [14]:
from sklearn.metrics import classification_report

print (classification_report(y,al.predict(X)))

             precision    recall  f1-score   support

        0.0       0.60      0.98      0.75        57
        1.0       0.86      0.14      0.24        43

avg / total       0.71      0.62      0.53       100

